# Simulation

## 9.1 Introduction

- Baseball's nested structure: seasons made of games, games of innings, innings of plate appearances lends itself well to simulation. 
- This chapter covers simulating half-innings with Markov chains and full seasons using the Bradley-Terry model.

## 9.2 Simulating a Half Inning

### 9.2.1 Markov chains

- A Markov chain models transitions between states, where each state describes runners on base and outs 
    - 24 possible combinations, plus the inning-ending 3 out state. 
- The key assumption is that the probability of moving to a new state depends only on the current state, not what happened earlier in the inning.

### 9.2.2 Review of work in run expectancy

Using Retrosheet play-by-play data, we can calculate how many runs are expected to score from each runners-and-outs state. These empirical frequencies form the foundation for our transition matrix.

In [1]:
import pandas as pd
import sys
sys.path.append('../src')
from retrosheet_utils import retrosheet_add_states

fields = pd.read_csv('../data/fields.csv')
headers = fields['Header'].str.lower().tolist()

retro2016 = pd.read_csv('../data/all2016.csv', names=headers, low_memory=False)
retro2016 = retrosheet_add_states(retro2016)

- `state` - gives the runner locations and the number of outs at the beginning of each play.
- `new_state` - contains same information at the conclusion of the play.

In [2]:
retro2016['runs'] = retro2016['away_score_ct'] + retro2016['home_score_ct']
retro2016['half_inning_id'] = (retro2016['game_id'] + ' ' + 
                               retro2016['inn_ct'].astype(str) + ' ' + 
                               retro2016['bat_home_id'].astype(str))

half_innings = (retro2016
    .groupby('half_inning_id')
    .agg(
        outs_inning=('event_outs_ct', 'sum'),
        runs_inning=('runs_scored', 'sum'),
        runs_start=('runs', 'first')
    )
    .reset_index()
)

half_innings['max_runs'] = half_innings['runs_inning'] + half_innings['runs_start']

- Create the variable `half_inning_id` as a unique identifier for each half-inning in each baseball game.
- `half_innings` new dataframe that contains data aggregated over each half-inning of baseball played in 2016.

In [3]:
retro2016 = retro2016.merge(half_innings, on='half_inning_id', how='inner')

retro2016_complete = retro2016[
    ((retro2016['state'] != retro2016['new_state']) | (retro2016['runs_scored'] > 0)) &
    (retro2016['outs_inning'] == 3) &
    (retro2016['bat_event_fl'] == 'T')
].copy()


In [4]:
retro2016_complete['new_state'] = retro2016_complete['new_state'].str.replace(
    r'[0-1]{3} 3', '3', regex=True
)

- `new_state` runner locations when there were three outs.
    - always have the value 3 when the number of outs is equal to 3

### 9.2.3 Computing the transition probabilities

Computing the frequencies of all possible transitions between states 

In [5]:
T_matrix = pd.crosstab(retro2016_complete['state'], retro2016_complete['new_state'])
T_matrix.shape

(24, 25)

There are 24 possible values of the beginning state `state`, and 25 values of the final state `new_state` including the 3-outs state.

Creating probability matrix:

In [6]:
P_matrix = T_matrix.div(T_matrix.sum(axis=1), axis=0)
P_matrix.shape

(24, 25)

In [7]:
P_matrix.loc['3'] = [0] * 24 + [1]

In [8]:
P_matrix.shape

(25, 25)

The matrix `P_matrix` now has two important properties that allow it to model transitions between states in a Markov chain 
   1. it is square
   2. the entries in each of its rows sum to 1.

In [9]:
P_matrix.sum(axis=1)

state
000 0    1.0
000 1    1.0
000 2    1.0
001 0    1.0
001 1    1.0
001 2    1.0
010 0    1.0
010 1    1.0
010 2    1.0
011 0    1.0
011 1    1.0
011 2    1.0
100 0    1.0
100 1    1.0
100 2    1.0
101 0    1.0
101 1    1.0
101 2    1.0
110 0    1.0
110 1    1.0
110 2    1.0
111 0    1.0
111 1    1.0
111 2    1.0
3        1.0
dtype: float64

In [10]:
(P_matrix.loc[['000 0']]
    .T
    .reset_index()
    .rename(columns={'index': 'new_state', '000 0': 'Prob'})
    .query('Prob > 0')
)

state,new_state,Prob
0,000 0,0.033364
1,000 1,0.675935
3,001 0,0.005627
6,010 0,0.050267
12,100 0,0.234807


In [11]:
(P_matrix.loc[['010 2']]
    .T
    .reset_index()
    .rename(columns={'index': 'new_state', '010 2': 'Prob'})
    .query('Prob > 0')
)

state,new_state,Prob
2,000 2,0.023319
5,001 2,0.005867
8,010 2,0.057620
11,011 2,0.000451
14,100 2,0.074470
17,101 2,0.032496
20,110 2,0.155709
24,3,0.650068


###  9.2.4 Simulating the Markov chain

The number of runs scored on a transition equals the sum of runners and outs before the play, minus the sum after, plus one (for the batter):

$$\text{runs} = (N_{runners}^{before} + O^{before} + 1) - (N_{runners}^{after} + O^{after})$$

In [12]:
def num_havent_scored(s):
    return sum(int(c) for c in s if c.isdigit())

runners_out = {state: num_havent_scored(state) for state in T_matrix.index}
runners_out = pd.Series(runners_out)

In [13]:
import numpy as np

R_runs = np.subtract.outer(runners_out.values + 1, runners_out.values)
R_runs = pd.DataFrame(R_runs, index=runners_out.index, columns=runners_out.index)
R_runs['3'] = 0


The simulation randomly samples state transitions using the probability matrix until reaching 3 outs, accumulating runs along the way.

In [14]:
def simulate_half_inning(P, R, start=0):
    s = start
    runs = 0
    while s < 24:  # 24 is the "3" absorbing state index
        s_new = np.random.choice(25, p=P.iloc[s].values)
        runs += R.iloc[s, s_new]
        s = s_new
    return runs


In [15]:
np.random.seed(111653)
simulated_runs = [simulate_half_inning(P_matrix, R_runs) for _ in range(10000)]

In [16]:
pd.Series(simulated_runs).value_counts().sort_index()

0    7245
1    1473
2     729
3     323
4     131
5      53
6      24
7      17
8       2
9       3
Name: count, dtype: int64

**Key findings from 10,000 simulated half-innings:**
- ~73% of innings produce zero runs
- Less than 1% produce 5+ runs  
- Average runs per half-inning ≈ 0.5

In [17]:
sum(np.array(simulated_runs) >= 5) / 10000

np.float64(0.0099)

In [18]:
np.mean(simulated_runs)

np.float64(0.4995)

In [19]:
def runs_j(j):
    return np.mean([simulate_half_inning(P_matrix, R_runs, j) for _ in range(10000)])

erm_2016_mc = pd.DataFrame({
    'state': P_matrix.index[:24],
    'mean_run_value': [runs_j(j) for j in range(24)]
})

erm_2016_mc['bases'] = erm_2016_mc['state'].str[:3]
erm_2016_mc['outs_ct'] = erm_2016_mc['state'].str[4].astype(int)
erm_2016_mc = erm_2016_mc.drop(columns='state')

erm_2016_mc.pivot(index='bases', columns='outs_ct', values='mean_run_value')

outs_ct,0,1,2
bases,,,
000,0.4672,0.2533,0.0990
001,1.3139,0.9218,0.3236
010,1.0939,0.6433,0.3015
011,1.9009,1.3133,0.4937
100,0.8413,0.5094,0.2181
101,1.7102,1.1466,0.4542
110,1.3793,0.8608,0.4153
111,2.1934,1.4757,0.6853


**Validating the model:** Simulate 10,000 half-innings from each starting state and compare the mean runs to the empirical run expectancy matrix from Chapter 5.

In [20]:
erm_2016 = pd.read_pickle('../data/erm_2016.pkl')

comparison = erm_2016.merge(erm_2016_mc, on=['bases', 'outs_ct'], suffixes=('_emp', '_mc'))
comparison['run_value_diff'] = (comparison['mean_run_value_emp'] - comparison['mean_run_value_mc']).round(2)

comparison[['bases', 'outs_ct', 'run_value_diff']].pivot(
    index='bases', 
    columns='outs_ct', 
    values='run_value_diff'
)

outs_ct,0,1,2
bases,,,
000,0.03,0.01,0.01
001,0.03,0.02,0.05
010,0.04,0.03,0.01
011,0.03,0.04,0.05
100,0.02,0.00,0.00
101,0.01,0.05,0.02
110,0.07,0.06,-0.00
111,-0.09,0.06,0.01


Small differences (within ±0.1 runs) confirm the Markov chain simulation closely matches empirical run expectancy.

### 9.2.5 Beyond run expectancy

Matrix powers of P reveal multi-step transition probabilities. P³ gives the probability of reaching each state after exactly 3 plate appearances.

In [21]:
P_matrix_3 = P_matrix @ P_matrix @ P_matrix

In [22]:
P_sorted = (P_matrix_3.loc['000 0']
    .reset_index()
    .rename(columns={'index': 'new_state', '000 0': 'Prob'})
    .sort_values('Prob', ascending=False)
)

P_sorted.head(6)

,new_state,Prob
24,3,0.372300
14,100 2,0.240896
19,110 1,0.081480
8,010 2,0.073897
2,000 2,0.052901
5,001 2,0.028618


The **fundamental matrix** N = (I - Q)⁻¹ gives the expected number of visits to each transient state before absorption (3 outs).

In [23]:
Q = P_matrix.iloc[:-1, :-1]
N = np.linalg.inv(np.eye(24) - Q)
N = pd.DataFrame(N, index=Q.index, columns=Q.columns)

In [24]:
N_0000 = N.loc['000 0'].round(2)
N_0000.head(6)

new_state
000 0    1.05
000 1    0.75
000 2    0.60
001 0    0.01
001 1    0.03
001 2    0.05
Name: 000 0, dtype: float64

In [25]:
round(sum(N_0000), 2)

4.27

Summing a row of N gives the expected plate appearances remaining in the inning from that starting state.

In [26]:
avg_num_plays = N.sum(axis=1).round(2)
avg_num_plays.head(8)

state
000 0    4.27
000 1    2.87
000 2    1.46
001 0    4.33
001 1    2.99
001 2    1.53
010 0    4.34
010 1    2.93
dtype: float64

Starting with bases empty and 0 outs, expect ~4.27 batters per half-inning. With 2 outs, only ~1.5 batters remain.

### 9.2.6 Transition probabilities for individual teams

Team-specific transition probabilities reflect batting strength, but individual teams have limited observations for rare transitions. We address this with **shrinkage estimation**.

In [27]:
retro2016_complete['home_team_id'] = retro2016_complete['game_id'].str[:3]
retro2016_complete['batting_team'] = np.where(
    retro2016_complete['bat_home_id'] == 0,
    retro2016_complete['away_team_id'],
    retro2016_complete['home_team_id']
)

In [28]:
T_team = (retro2016_complete
    .groupby(['batting_team', 'state', 'new_state'])
    .size()
    .reset_index(name='n')
)

In [29]:
T_team[T_team['batting_team'] == 'ANA'].head(6)

,batting_team,state,new_state,n
0,ANA,000 0,000 0,40
1,ANA,000 0,000 1,1007
2,ANA,000 0,001 0,9
3,ANA,000 0,010 0,75
4,ANA,000 0,100 0,359
5,ANA,000 1,000 1,31


In [30]:
T_team_S = (retro2016_complete[retro2016_complete['state'] == '100 2']
    .groupby(['batting_team', 'state', 'new_state'])
    .size()
    .reset_index(name='n')
)

T_team_S.sample(6)

,batting_team,state,new_state,n
127,NYN,100 2,000 2,17
153,PIT,100 2,3,240
23,BAL,100 2,100 2,1
209,WAS,100 2,100 2,1
185,TBA,100 2,001 2,1
207,WAS,100 2,010 2,8


In [31]:
T_WAS = T_team_S[T_team_S['batting_team'] == 'WAS'].copy()
T_WAS['p'] = T_WAS['n'] / T_WAS['n'].sum()

T_all = (retro2016_complete[retro2016_complete['state'] == '100 2']
    .groupby('new_state')
    .size()
    .reset_index(name='n')
)
T_all['p'] = T_all['n'] / T_all['n'].sum()

**Shrinkage formula** combines team-specific and league-wide probabilities:

$$\hat{p}_{EST} = \frac{n}{n+K} \cdot p_{TEAM} + \frac{K}{n+K} \cdot p_{ALL}$$

Where `K=1274` is a smoothing constant. Small sample sizes weight toward league average; large samples trust team data.

In [32]:
T_compare = T_WAS.merge(T_all, on='new_state', suffixes=('_team', '_all'))
T_compare['p_EST'] = (
    (T_compare['n_team'] / (1274 + T_compare['n_team'])) * T_compare['p_team'] +
    (1274 / (1274 + T_compare['n_team'])) * T_compare['p_all']
)
T_compare[['batting_team', 'new_state', 'p_team', 'p_all', 'p_EST']]

,batting_team,new_state,p_team,p_all,p_EST
0,WAS,000 2,0.031915,0.029098,0.029124
1,WAS,001 2,0.005319,0.005768,0.005767
2,WAS,010 2,0.021277,0.021952,0.021948
3,WAS,011 2,0.021277,0.021952,0.021948
4,WAS,100 2,0.002660,0.000775,0.000776
5,WAS,101 2,0.045213,0.043475,0.043497
6,WAS,110 2,0.183511,0.194645,0.194073
7,WAS,3,0.688830,0.682335,0.683432


The smoothed estimates (`p_EST`) lie between team and league values, pulling extreme team estimates toward the league mean especially for rare transitions with small n.

## 9.3 Simulating a Baseball Season

### 9.3.1 The Bradley-Terry model

The Bradley-Terry model assigns each team a talent value T drawn from N(0, σ). The probability that team A defeats team B:

$$P(A \text{ wins}) = \frac{e^{T_A}}{e^{T_A} + e^{T_B}}$$

This is equivalent to Bill James's log5 method, where talent equals log-odds of winning. A team with T=0 wins ~50%; T=0.2 yields ~55% wins.

### 9.3.2 Making up a schedule

Using 1968 as model season: 10 teams per league, each playing 18 games against every opponent (9 home, 9 away), no interleague play.

In [33]:
def make_schedule(teams, k):
    num_teams = len(teams)
    home = [team for team in teams for _ in range(num_teams)] * k
    visitor = (teams * num_teams) * k
    schedule = pd.DataFrame({'Home': home, 'Visitor': visitor})
    return schedule[schedule['Home'] != schedule['Visitor']]

In [34]:
teams = pd.read_csv('../data/lahman/Teams.csv')

teams_68 = teams[teams['yearID'] == 1968][['teamID', 'lgID']].copy()
teams_68['teamID'] = teams_68['teamID'].astype(str)

schedule = pd.concat([
    make_schedule(group['teamID'].tolist(), k=9).assign(lgID=name)
    for name, group in teams_68.groupby('lgID')
])

schedule.shape

(1620, 3)

1,620 total games: 10 teams × 9 opponents × 9 home games × 2 leagues = 810 per league.

### 9.3.3 Simulating talents and computing win probabilities

Assign talents to each team and join to the schedule to compute game-by-game win probabilities.

In [35]:
s_talent = 0.20
np.random.seed(42)
teams_68['talent'] = np.random.normal(0, s_talent, len(teams_68))

schedule_talent = (schedule
    .merge(teams_68[['lgID', 'teamID', 'talent']], 
           left_on=['lgID', 'Home'], right_on=['lgID', 'teamID'])
    .rename(columns={'talent': 'talent_home'})
    .drop(columns='teamID')
    .merge(teams_68[['lgID', 'teamID', 'talent']], 
           left_on=['lgID', 'Visitor'], right_on=['lgID', 'teamID'])
    .rename(columns={'talent': 'talent_visitor'})
    .drop(columns='teamID')
)

In [36]:
schedule_talent['prob_home'] = (
    np.exp(schedule_talent['talent_home']) / 
    (np.exp(schedule_talent['talent_home']) + np.exp(schedule_talent['talent_visitor']))
)

In [37]:
schedule_talent.head(6)

,Home,Visitor,lgID,talent_home,talent_visitor,prob_home
0,BAL,BOS,AL,-0.027653,0.129538,0.460783
1,BAL,CAL,AL,-0.027653,0.304606,0.417691
2,BAL,CHA,AL,-0.027653,-0.046831,0.504794
3,BAL,CLE,AL,-0.027653,0.153487,0.454838
4,BAL,DET,AL,-0.027653,-0.093895,0.516554
5,BAL,MIN,AL,-0.027653,-0.093146,0.516367


Win probabilities vary based on talent difference. Teams with higher talent have >50% chance against weaker opponents.

###  9.3.4 Simulating the regular season

In [38]:
schedule_talent['outcome'] = np.random.binomial(1, schedule_talent['prob_home'])
schedule_talent['winner'] = np.where(
    schedule_talent['outcome'] == 1,
    schedule_talent['Home'],
    schedule_talent['Visitor']
)

In [39]:
schedule_talent[['Visitor', 'Home', 'prob_home', 'outcome', 'winner']].head(6)

,Visitor,Home,prob_home,outcome,winner
0,BOS,BAL,0.460783,0,BOS
1,CAL,BAL,0.417691,1,BAL
2,CHA,BAL,0.504794,1,BAL
3,CLE,BAL,0.454838,0,CLE
4,DET,BAL,0.516554,0,DET
5,MIN,BAL,0.516367,1,BAL


In [40]:
results = (schedule_talent
    .groupby('winner')
    .size()
    .reset_index(name='Wins')
    .merge(teams_68, left_on='winner', right_on='teamID')
)

results

,winner,Wins,teamID,lgID,talent
0,ATL,83,ATL,NL,0.099343
1,BAL,77,BAL,AL,-0.027653
2,BOS,90,BOS,AL,0.129538
3,CAL,99,CAL,AL,0.304606
4,CHA,78,CHA,AL,-0.046831
5,CHN,73,CHN,NL,-0.046827
6,CIN,91,CIN,NL,0.315843
7,CLE,96,CLE,AL,0.153487
8,DET,74,DET,AL,-0.093895
9,HOU,93,HOU,NL,0.108512


### 9.3.5 Simulating the post-season

In [41]:
def win_league(res):
    res = res.copy()
    res['tiebreaker'] = np.random.uniform(size=len(res))
    res['wins_total'] = res['Wins'] + res['tiebreaker']
    res['rank'] = res.groupby('lgID')['wins_total'].rank(ascending=False, method='min')
    res['is_winner_lg'] = res.groupby('lgID')['wins_total'].transform('max') == res['wins_total']
    return res

In [42]:
sim_one = win_league(results)

# Get league winners and simulate World Series (best of 7)
league_winners = sim_one[sim_one['is_winner_lg']].copy()

talents = league_winners['talent'].values
probs = np.exp(talents) / np.exp(talents).sum()
outcomes = np.random.multinomial(7, probs)
league_winners['outcome'] = outcomes
league_winners['is_winner_ws'] = league_winners['outcome'] > 3

ws_winner = league_winners[league_winners['is_winner_ws']][['winner', 'is_winner_ws']]

# Join back to full results
sim_one = sim_one.merge(ws_winner, on='winner', how='left')
sim_one['is_winner_ws'] = sim_one['is_winner_ws'].isna() == False

sim_one

,winner,Wins,teamID,lgID,talent,tiebreaker,wins_total,rank,is_winner_lg,is_winner_ws
0,ATL,83,ATL,NL,0.099343,0.089124,83.089124,4.0,False,False
1,BAL,77,BAL,AL,-0.027653,0.755270,77.755270,6.0,False,False
2,BOS,90,BOS,AL,0.129538,0.127713,90.127713,3.0,False,False
3,CAL,99,CAL,AL,0.304606,0.826068,99.826068,1.0,True,False
4,CHA,78,CHA,AL,-0.046831,0.782028,78.782028,5.0,False,False
5,CHN,73,CHN,NL,-0.046827,0.708745,73.708745,10.0,False,False
6,CIN,91,CIN,NL,0.315843,0.036160,91.036160,2.0,False,False
7,CLE,96,CLE,AL,0.153487,0.303128,96.303128,2.0,False,False
8,DET,74,DET,AL,-0.093895,0.263113,74.263113,7.0,False,False
9,HOU,93,HOU,NL,0.108512,0.360136,93.360136,1.0,True,True
